<a href="https://colab.research.google.com/github/xuanmse/python-for-engineer/blob/master/Assignment_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Yêu cầu**

Cho tập dữ liệu về doanh số bán hàng gồm các thông tin như sau (https://drive.google.com/file/d/1QWD8rceAWo6KJYHh6Tx0eBtAM9U1EQno/view?usp=sharing(sad)
Superstore Sales Dataset Analysis | Kaggle
```
- order_id
- order_date
- ship_date
- ship_mode
- customer_id
- customer_name
- segment
- country
- city
- state
- region
- product_id
- sub-category
- product_name
- sales
```
Hãy sử dụng Python và Jupyter Notebook để trả lời các câu hỏi sau:
- YC1: Tiền xử lý dữ liệu (xử lý các trường hợp thiếu dữ liệu, chuẩn hóa ngày tháng, đơn vị,...).
- YC2: Vẽ biểu đồ thể hiện biến động doanh số theo tháng. Cho biết doanh thu trung bình trong 3 tháng gần nhất.
- YC3: Vẽ biểu đồ doanh số bán hàng theo quý và cho biết quý nào có tổng doanh số cao nhất?
- YC4: Tính thời gian trung giao hàng trung bình (ship date - order date) theo từng tháng và vẽ biểu đồ mô tả sự biến thiên của con số này.
- YC5: Vẽ biểu đồ thể hiện sức mua theo từng thành phố. Cho biết top 3 thành phố có doanh số thấp nhất và 3 thành phố có doanh số cao nhất.
- YC6: Vẽ biểu đồ thể hiện tỉ trọng doanh số theo bang (state) và cho biết trung bình doanh số của 5 bang dẫn đầu về sức mua.
- YC7: Cho biết các bang dẫn đầu doanh số theo từng vùng (east, west, south, central,..).
- YC8: Cho biết top 3 sản phẩm có doanh thu cao nhất và top 3 sản phẩm có số lượng đơn vị bán ra cao nhất.
- YC9: Vẽ biểu đồ thể hiện tỉ trọng doanh số của từng category.
- YC10: Dự báo doanh số theo nhiều khía cạnh khác nhau trong tương lai: theo vùng, theo bang, theo category,..  và phát hành API để khai thác.

#**Cài đặt môi trường & tải tập dữ liệu**

##**Cài đặt thư viện**

In [ ]:
!pip install kaggle # cài đặt công cụ Kaggle
!pip install seaborn
!pip install plotly

##**Cài đặt xác thực**

In [ ]:
# Tải thông tin tài khoảng để xác thực trên kaggle và lưu vào máy ảo colab
! yes | mkdir /root/.kaggle
! yes | wget https://raw.githubusercontent.com/xuanmse/python-for-engineer/master/Asessment1/kaggle.json -P /root/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

##**Giải nén**

In [ ]:
!kaggle datasets download -d rohitsahoo/sales-forecasting -p /kaggle/input/
# !yes | kaggle competitions download -c m5-forecasting-accuracy -p /kaggle/input/
!yes | unzip /kaggle/input/sales-forecasting.zip -d /kaggle/input/sales-forecasting
!yes | rm /kaggle/input/sales-forecasting.zip

##**Nhập thư viện và kiểm tra dữ liệu**

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split


##**Tải tập dữ liệu lên ô nhớ**

In [ ]:
# Reading dataset's ...
train = pd.read_csv('/kaggle/input/sales-forecasting/train.csv')

##**Kiểm tra dữ liệu đã tải lên**

In [ ]:
train.info()

#**YC1: Tiền xử lý dữ liệu**

##**Kiểm tra dữ liệu rổng**

In [ ]:
train.isnull().sum().sort_values(ascending = False)

Thiếu dữ liệu về postal code, sử dụng thông tin thành phố để điền cập nhật lại

##**Cập nhật dữ liệu post code**

In [ ]:
train[train['Postal Code'].isnull()==True]['City'].unique()

In [ ]:
train[train['City']=='Burlington']['Postal Code'].value_counts()
train['Postal Code'] = train['Postal Code'].fillna(27217)
train['Postal Code'] = train['Postal Code'].apply(int)
train.isnull().sum().sort_values(ascending = False)

In [ ]:
train

##**Cập nhật tên cột**


In [ ]:
train.rename(columns={'Row ID':'Row_ID',
                   'Order ID': 'Order_ID',
                   'Order Date':'Order_Date',
                   'Ship Date':'Ship_Date',
                   'Customer ID':'Custmoer_ID',
                   'Customer Name':'Customer_Name',
                   'Postal Code':'Postal_Code',
                   'Product ID':'Product_ID',
                   'Sub-Category':'Sub_Category',
                   'Product Name':'Product_Name'},inplace=True)
train.info()

##**Cập nhận dữ liệu ngày tháng**

In [ ]:
train['Order_Date'] = pd.to_datetime(train['Order_Date'], dayfirst=True)
train['Ship_Date'] = pd.to_datetime(train['Ship_Date'],dayfirst=True)
train

#**YC2: Vẽ biểu đồ thể hiện biến động doanh số theo tháng. Cho biết doanh thu trung bình trong 3 tháng gần nhất.**

Tính toán dữ liệu

In [ ]:
g = train
per = train.Order_Date.dt.to_period("M")
g = g.groupby(per)["Sales"].sum()
g

In [ ]:
fig = px.bar(g, x=g.index.strftime('%b-%Y'),y = "Sales")
fig.show()

In [ ]:
fig = px.line(g, x=g.index.strftime('%b-%Y'),y = "Sales")
fig.show()

Doanh thu trung bình trong 3 tháng gần nhất.

In [ ]:
g = train.groupby(per)["Sales"].mean()
g[-3:]

#**YC3: Vẽ biểu đồ doanh số bán hàng theo quý và cho biết quý nào có tổng doanh số cao nhất?**

In [ ]:
per = train.Order_Date.dt.to_period("Q")
g = train
g = g.groupby(per)["Sales"].sum()
g

In [ ]:
fig = px.bar(g, x=g.index.strftime('%q-%Y'),y = "Sales")
fig.show()

Quý có doanh thu cao nhất:

In [ ]:
# Quý có doanh số cao nhất
F  = g.sort_values(ascending=False)
print(F.index[0], F.values[0])
# fig = px.bar(g, x=g.index.to_timestamp(),y = "Row_ID")
# fig.show()

#**YC4: Tính thời gian trung giao hàng trung bình (ship date - order date) theo từng tháng và vẽ biểu đồ mô tả sự biến thiên của con số này.**


In [ ]:
X = train 
X["Deliver_time"] = X["Ship_Date"] - X["Order_Date"] 

per = X.Order_Date.dt.to_period("M")
g = X.groupby(per)["Deliver_time"].mean()/ np.timedelta64(1, 'D') # thời gian theo giờ
g


In [ ]:
fig = px.bar(g, x=g.index.strftime('%b-%Y'),y = "Deliver_time")
fig.show()

#**YC5: Vẽ biểu đồ thể hiện sức mua theo từng thành phố. Cho biết top 3 thành phố có doanh số thấp nhất và 3 thành phố có doanh số cao nhất.**

In [ ]:
g = train
g = g.groupby(["City"])["Sales"].sum()
g

In [ ]:
fig = px.bar(g)
fig.show()

In [ ]:
g  = g.sort_values(ascending=False)
print(g.head(3))

In [ ]:
print(g.tail(3))

#**YC6: Vẽ biểu đồ thể hiện tỉ trọng doanh số theo bang (state) và cho biết trung bình doanh số của 5 bang dẫn đầu về sức mua.**

In [ ]:
g = train
fig = px.pie(g, values='Sales', names='State')
fig.show()

In [ ]:
g = g.groupby(["State"])["Sales"].sum().sort_values(ascending=False)

In [ ]:
g.head(5).mean()

#**YC7: Cho biết các bang dẫn đầu doanh số theo từng vùng (east, west, south, central,..).**

In [ ]:
g = train
g.groupby(["Region","State"])["Sales"].sum().sort_values().groupby(level=0).tail(1)

#**YC8: Cho biết top 3 sản phẩm có doanh thu cao nhất và top 3 sản phẩm có số lượng đơn vị bán ra cao nhất.**

Top3 sản phẩm có doanh thu cao nhất

In [ ]:
g = train
g.groupby("Product_ID")["Sales"].sum().sort_values(ascending=False).head(3)

Top 3 sản phẩm có đơn vị bán ra cao nhất

In [ ]:
g = train
g.groupby("Product_ID")["Row_ID"].sum().sort_values(ascending=False).head(3)

#**YC9: Vẽ biểu đồ thể hiện tỉ trọng doanh số của từng category.**

In [ ]:
g = train
g

In [ ]:
fig = px.pie(g, values='Sales', names='Category')
fig.show()

#**YC10: Dự báo doanh số theo nhiều khía cạnh khác nhau trong tương lai: theo vùng, theo bang, theo category,.. và phát hành API để khai thác.**

Dự báo danh số bán hàng theo bang cho các ngày tiếp theo

In [ ]:
train

In [ ]:
def encode_dates(df, column):
    df = df.copy()
    df[column] = pd.to_datetime(df[column])
    df[column + '_year'] = df[column].apply(lambda x: x.year)
    df[column + '_month'] = df[column].apply(lambda x: x.month)
    df[column + '_day'] = df[column].apply(lambda x: x.day)
    df = df.drop(column, axis=1)
    return df

# Features

target_col = 'Sales'

# exclude_cols = ['Row_ID','Order_ID','Ship Mode','Custmoer_ID','Customer_Name', 'Country', 'Deliver_time',"Order_ID",	"Ship_Date",	"Ship Mode",	"Custmoer_ID",	"Customer_Name", "Segment","Product_Name","State","Postal_Code","Region", "Product_ID","Category","Sub_Category"	]
# feature_cols = [col for col in train.columns if col not in exclude_cols]

# train_data[feature_cols]

df = train.copy()
df = encode_dates(df, column='Order_Date')

df = df.groupby(["State","Order_Date_year","Order_Date_month","Order_Date_day"])["Sales"].sum()
df
# df.index["1"]
# df["Order_Date"] =  df["Order_Date"].strftime('%Y-%M-%D')

# 
# train_data = train.groupby([])

In [ ]:
X = pd.get_dummies(df.index.to_frame())
Y = df.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, shuffle=True, random_state=1)
X.info()

In [ ]:
import tensorflow as tf
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Dense(256, activation='relu')(inputs)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())

In [ ]:
model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

In [ ]:
test_loss = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss: {:.5f}".format(test_loss))

In [ ]:
from sklearn.metrics import r2_score
y_pred = np.squeeze(model.predict(X_test))
test_r2 = r2_score(y_test, y_pred)

print("Test R^2 Score: {:.5f}".format(test_r2))

In [ ]:
len(X_test.columns)

In [ ]:
cus_test_state = "Alabama"
cus_test_year = 2016
cus_test_month = 4
cus_test_day  = 7
key_test = ("Alabama",2015,4,7)
# key_input = X_test.iloc[[2]].copy()
# key_input
key_test =pd.MultiIndex.from_tuples([(cus_test_state,cus_test_year,cus_test_month,cus_test_day)], names=["State","Order_Date_year", "Order_Date_month", "Order_Date_day"])
X_test.columns
ff = np.zeros(52,int)

data_input = pd.DataFrame([ff],index=[key_test], columns = X_test.columns)
data_input.iloc[0] = int(0)
data_input["Order_Date_year"] = cus_test_year
data_input["Order_Date_month"] = cus_test_month
data_input["Order_Date_day"] = cus_test_day
data_input["State_" + cus_test_state] = 1
data_input

model.predict(data_input)
# key_test_en = pd.get_dummies(key_test.to_frame())
# X_test.iloc[[2]]

# key_test_en
# df_key = pd.DataFrame(, columns=X_test.columns)
# model.predict(X_test.iloc[[2]])

In [ ]:
df.index